In [10]:
import geopandas as gpd
import pandas as pd
import folium
import branca
import requests
import json
import urllib.request

from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

In [2]:
print(folium.__version__)

0.11.0


In [17]:
map_data=pd.read_csv('map_data.csv')
global_data=map_data.loc[map_data['SettlementType']=='- Country Overview',:]
global_data_to_plot=global_data[global_data.Cases != 'TBC']
dict_to_json={
    'GREECE / COUNTRY OVERVIEW':'Greece',
    'LEBANON / COUNTRY OVERVIEW':'Lebanon',
    '🇺🇸UNITED STATES / COUNTRY OVERVIEW':'United States of America',
    'Afghanistan Overview':'Afghanistan'
}
global_data_to_plot['CountryName']=global_data_to_plot['Name'].apply(lambda x: dict_to_json[x])
global_data_to_plot=global_data_to_plot.astype({'Cases': 'int','CountryName':'string'})

/Users/billyzhaoyh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [18]:
global_data_to_plot

Name      SettlementType  Cases Population  \
0            GREECE / COUNTRY OVERVIEW  - Country Overview    162      80000   
1           LEBANON / COUNTRY OVERVIEW  - Country Overview      5    1500000   
63  🇺🇸UNITED STATES / COUNTRY OVERVIEW  - Country Overview    606      29675   
82                Afghanistan Overview  - Country Overview   1531        NaN   

     Latitude  Longitude               CountryName  
0   39.074208  21.824312                    Greece  
1   33.854721  35.862285                   Lebanon  
63  38.899391 -76.990552  United States of America  
82  33.939110  67.709953               Afghanistan

In [9]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_geo = f'{url}/world-countries.json'
import urllib.request, json 
with urllib.request.urlopen(country_geo) as url:
    data = json.loads(url.read().decode())

In [12]:
trimmed_data={}
country_codes=global_data_to_plot['Name'].apply(lambda x: dict_to_json[x]).tolist()
list_of_features=[]
for key,value in data.items():
    if key=='features':
        for country in value:
            if country['id'] in country_codes:
                list_of_features.append(country)
for key,value in data.items():
    if key=='features':
        trimmed_data['features']=list_of_features
    else:
        trimmed_data[key]=value

In [14]:
countries = gpd.GeoDataFrame.from_features(trimmed_data, crs='EPSG:4326')
countries.head()

geometry                      name
0  POLYGON ((61.21082 35.65007, 62.23065 35.27066...               Afghanistan
1  MULTIPOLYGON (((23.69998 35.70500, 24.24667 35...                    Greece
2  POLYGON ((35.82110 33.27743, 35.55280 33.26427...                   Lebanon
3  MULTIPOLYGON (((-155.54211 19.08348, -155.6881...  United States of America

In [19]:
countriesmerge = countries.merge(global_data_to_plot, how='left', left_on='name', right_on='CountryName')

In [20]:
countriesmerge

geometry  \
0  POLYGON ((61.21082 35.65007, 62.23065 35.27066...   
1  MULTIPOLYGON (((23.69998 35.70500, 24.24667 35...   
2  POLYGON ((35.82110 33.27743, 35.55280 33.26427...   
3  MULTIPOLYGON (((-155.54211 19.08348, -155.6881...   

                       name                                Name  \
0               Afghanistan                Afghanistan Overview   
1                    Greece           GREECE / COUNTRY OVERVIEW   
2                   Lebanon          LEBANON / COUNTRY OVERVIEW   
3  United States of America  🇺🇸UNITED STATES / COUNTRY OVERVIEW   

       SettlementType  Cases Population   Latitude  Longitude  \
0  - Country Overview   1531        NaN  33.939110  67.709953   
1  - Country Overview    162      80000  39.074208  21.824312   
2  - Country Overview      5    1500000  33.854721  35.862285   
3  - Country Overview    606      29675  38.899391 -76.990552   

                CountryName  
0               Afghanistan  
1                    Greece  
2                   Lebanon  
3  United States of America

In [21]:
colormap = branca.colormap.LinearColormap(
    vmin=countriesmerge['Cases'].quantile(0.0), 
    vmax=countriesmerge['Cases'].quantile(1), 
    colors=['red','orange','lightblue','green','darkgreen'],
    caption="Confirmed COVID cases of refugee/displaced/detained persons",
)

In [99]:
m = folium.Map(location=[36, -42], zoom_start=2)

# popup = GeoJsonPopup(
#     fields=['Name','Cases'],
#     aliases=['Country',"cases"],
#     localize=True,
#     labels=True,
#     style="background-color: yellow;",
# )

tooltip = GeoJsonTooltip(
    fields=["Name", "Cases", "Population"],
    aliases=["Country:", "No. of cases so far:", "Total population of refugee/displaced/detained:"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)


g = folium.GeoJson(
    countriesmerge,
    style_function=lambda x: {
        "fillColor": colormap(x["properties"]["Cases"])
        if x["properties"]["Cases"] is not None
        else "transparent",
        "color": "black",
        "fillOpacity": 0.4,
    },
    tooltip=tooltip,
#     popup=popup
).add_to(m)

colormap.add_to(m)

m

In [100]:
centre_data=map_data.loc[map_data['SettlementType']!='- Country Overview',:]

In [101]:
centre_data=centre_data.round({'Latitude': 2, 'Longitude': 2})

In [102]:
centre_data.SettlementType.unique()

array(['Boat Rescue', 'Detention center', 'Displaced Camp',
       'Displaced Camp, Refugee camp',
       'Displaced Camp, Urban Refugee Camp', 'Informal Camp',
       'Informal settlement', 'Migrant Shelter',
       'Migrant Shelter, Urban Refugees', 'Old Refugee Camp',
       'Old Refugee Camp, Urban Refugee Camp', 'Refugee camp',
       'Refugee camp, Urban Refugee Camp', 'Refugee center', 'Shelter',
       'Urban Refugees'], dtype=object)

In [103]:
colors=['red', 'blue', 'green', 'purple', 'orange', 'darkred','lightred', 'beige', 
 'darkblue', 'darkgreen', 'cadetblue','darkpurple', 'white', 'pink','lightgreen','gray', 'black', 'lightgray']

In [104]:
from folium.plugins import MarkerCluster

In [105]:
marker_cluster = MarkerCluster().add_to(m)

In [106]:
i=0
grouped = centre_data.groupby('SettlementType')
for name, group in grouped:
    for row in group.itertuples():
        folium.Marker(
        location=[row.Latitude, row.Longitude],
        popup=f'{row.Name} has {row.Cases} confirmed COVID-19 cases',
        icon=folium.Icon(color=colors[i],prefix='fa', icon='circle'),
        ).add_to(marker_cluster)
    i+=1

In [107]:
m

In [108]:
i=0
for name, group in grouped:
    print(name)
    print(colors[i])
    i+=1

Boat Rescue
red
Detention center
blue
Displaced Camp
green
Displaced Camp, Refugee camp
purple
Displaced Camp, Urban Refugee Camp
orange
Informal Camp
darkred
Informal settlement
lightred
Migrant Shelter
beige
Migrant Shelter, Urban Refugees
darkblue
Old Refugee Camp
darkgreen
Old Refugee Camp, Urban Refugee Camp
cadetblue
Refugee camp
darkpurple
Refugee camp, Urban Refugee Camp
white
Refugee center
pink
Shelter
lightgreen
Urban Refugees
gray


In [109]:
legend_html =   '''
                <div style="position: fixed; 
                            bottom: 50px; left: 50px; width: 220px; height: 350px; 
                            border:2px solid grey; z-index:9999; font-size:10px;
                            ">&nbsp; <b> Settlement Type </b> <br>
                              &nbsp; Boat Rescue &nbsp; <i class='fa fa-map-marker fa-2x' style='color:red'></i><br>
                              &nbsp; Detention center &nbsp; <i class='fa fa-map-marker fa-2x' style='color:blue'></i><br>
                              &nbsp; Displaced Camp &nbsp; <i class='fa fa-map-marker fa-2x' style='color:green'></i><br>
                              &nbsp; Displaced Camp, Refugee camp &nbsp; <i class='fa fa-map-marker fa-2x' style='color:purple'></i><br>
                              &nbsp; Displaced Camp, Urban Refugee Camp &nbsp; <i class='fa fa-map-marker fa-2x' style='color:orange'></i><br>
                              &nbsp; Informal Camp &nbsp; <i class='fa fa-map-marker fa-2x' style='color:darkred'></i><br>
                              &nbsp; Informal settlement &nbsp; <i class='fa fa-map-marker fa-2x' style='color:lightred'></i><br>
                              &nbsp; Migrant Shelter &nbsp; <i class='fa fa-map-marker fa-2x' style='color:beige'></i><br>
                              &nbsp; Migrant Shelter, Urban Refugees &nbsp; <i class='fa fa-map-marker fa-2x' style='color:darkblue'></i><br>
                              &nbsp; Old Refugee Camp &nbsp; <i class='fa fa-map-marker fa-2x' style='color:darkgreen'></i><br>
                              &nbsp; Old Refugee Camp, Urban Refugee Camp &nbsp; <i class='fa fa-map-marker fa-2x' style='color:cadetblue'></i><br>
                              &nbsp; Refugee camp &nbsp; <i class='fa fa-map-marker fa-2x' style='color:darkpurple'></i><br>
                              &nbsp; Refugee camp, Urban Refugee Camp &nbsp; <i class='fa fa-map-marker fa-2x' style='color:white'></i><br>
                              &nbsp; Refugee center &nbsp; <i class='fa fa-map-marker fa-2x' style='color:pink'></i><br>
                              &nbsp; Shelter &nbsp; <i class='fa fa-map-marker fa-2x' style='color:lightgreen'></i><br>
                              &nbsp; Urban Refugees &nbsp; <i class='fa fa-map-marker fa-2x' style='color:gray'></i>
                </div>
                ''' 

In [110]:
m.get_root().html.add_child(folium.Element(legend_html))

In [112]:
m.save('index.html')

In [113]:
import codecs
f=codecs.open("notion_test.html", 'r')
notion=f.read()

In [116]:
iframe = folium.IFrame(html=notion, width=500, height=300)
popup = folium.Popup(iframe, max_width=2650)

folium.Marker([30,-100], popup=popup).add_to(m)

In [117]:
m

In [118]:
m.save('notion.html')